we use word embeding method to produce word translation from english to french. as word embeddings are large file i just little pickle file eith specific words.
for English word embeding we can use[archive word2vec, GoogleNews-vectors-negative300.bin.gz](https://code.google.com/archive/p/word2vec/)
for french we can use this path:[cross_lingual_text_classification]( https://github.com/vjstark/crosslingual_text_classification)
in the terminal, type (in one line) curl -o ./wiki.multi.fr.vec https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.fr.vec

you can run these lines to produce pickle file. i use pickle files from coursera natural language processing specialization 

In [2]:
!pip install nltk
!pip install gensim
!pip install sklearn
!pip install pandas

     |████████████████████████████████| 1.4 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 949 kB/s eta 0:00:01
     |████████████████████████████████| 723 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 1.1 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1435513 sha256=d4ba95ac0e71603513e543e76dc3fc46ffadeab8107410534ee1924dbea77304
  Stored in directory: /root/.cache/pip/wheels/de/5e/42/64abaeca668161c3e2cecc24f864a8fc421e3d07a104fc8a51
Successfully built nltk
     |████████████████████████████████| 24.2 MB 43 kB/s eta 0:00:012
     |████████████████████████████████| 117 kB 37 kB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-4.0.1-py3-none-any.whl size=115166 sha256=49d6ba1f7249ff6cff9f7d57981883c2113ec8fce83a64b31b25899020077a71
  Stored in directory: /root/.cache/pip/wheels/8b/5e/70/42adcaea93c80417ec6accf7db1d6d02367ed02f2254cd5eef
Successfully built smart-open
You should consider u

In [13]:
import pdb
import pickle
import string

import time

import gensim
import matplotlib.pyplot as plt
import nltk
import numpy as np
import scipy
import sklearn
from gensim.models import KeyedVectors
from nltk.corpus import stopwords, twitter_samples
from nltk.tokenize import TweetTokenizer
from os import getcwd
import pandas as pd

In [ ]:
# Use this code to download and process the full dataset on your local computer

# from gensim.models import KeyedVectors

# en_embeddings = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary = True)
# fr_embeddings = KeyedVectors.load_word2vec_format('./wiki.multi.fr.vec')


# # loading the english to french dictionaries
# en_fr_train = get_dict('en-fr.train.txt')
# print('The length of the english to french training dictionary is', len(en_fr_train))
# en_fr_test = get_dict('en-fr.test.txt')
# print('The length of the english to french test dictionary is', len(en_fr_train))

# english_set = set(en_embeddings.vocab)
# french_set = set(fr_embeddings.vocab)
# en_embeddings_subset = {}
# fr_embeddings_subset = {}
# french_words = set(en_fr_train.values())

# for en_word in en_fr_train.keys():
#     fr_word = en_fr_train[en_word]
#     if fr_word in french_set and en_word in english_set:
#         en_embeddings_subset[en_word] = en_embeddings[en_word]
#         fr_embeddings_subset[fr_word] = fr_embeddings[fr_word]


# for en_word in en_fr_test.keys():
#     fr_word = en_fr_test[en_word]
#     if fr_word in french_set and en_word in english_set:
#         en_embeddings_subset[en_word] = en_embeddings[en_word]
#         fr_embeddings_subset[fr_word] = fr_embeddings[fr_word]


# pickle.dump( en_embeddings_subset, open( "en_embeddings.p", "wb" ) )
# pickle.dump( fr_embeddings_subset, open( "fr_embeddings.p", "wb" ) )

### from now i use small pickle files

In [6]:
en_embeddings_subset = pickle.load(open("../data/en_embeddings.p", "rb"))
fr_embeddings_subset = pickle.load(open("../data/fr_embeddings.p", "rb"))

### look at the an la in english and french

In [7]:
en_embeddings_subset['the']

array([ 0.08007812,  0.10498047,  0.04980469,  0.0534668 , -0.06738281,
       -0.12060547,  0.03515625, -0.11865234,  0.04394531,  0.03015137,
       -0.05688477, -0.07617188,  0.01287842,  0.04980469, -0.08496094,
       -0.06347656,  0.00628662, -0.04321289,  0.02026367,  0.01330566,
       -0.01953125,  0.09277344, -0.171875  , -0.00131989,  0.06542969,
        0.05834961, -0.08251953,  0.0859375 , -0.00318909,  0.05859375,
       -0.03491211, -0.0123291 , -0.0480957 , -0.00302124,  0.05639648,
        0.01495361, -0.07226562, -0.05224609,  0.09667969,  0.04296875,
       -0.03540039, -0.07324219,  0.03271484, -0.06176758,  0.00787354,
        0.0035553 , -0.00878906,  0.0390625 ,  0.03833008,  0.04443359,
        0.06982422,  0.01263428, -0.00445557, -0.03320312, -0.04272461,
        0.09765625, -0.02160645, -0.0378418 ,  0.01190186, -0.01391602,
       -0.11328125,  0.09326172, -0.03930664, -0.11621094,  0.02331543,
       -0.01599121,  0.02636719,  0.10742188, -0.00466919,  0.09

In [8]:
fr_embeddings_subset['la']

array([-6.18250e-03, -9.43867e-04, -8.82648e-03,  3.24623e-02,
       -2.18281e-02,  2.98912e-02,  2.29990e-02,  2.80628e-02,
        5.87757e-03, -2.56806e-02,  4.21732e-02, -9.37519e-03,
        2.86577e-02, -5.08309e-02, -7.13388e-02, -8.34270e-03,
       -2.88709e-02, -3.85773e-02,  4.38649e-02,  1.18239e-01,
       -8.57782e-02,  2.65617e-02, -1.21373e-01, -1.88254e-02,
       -5.46811e-02, -8.06788e-02,  3.16896e-02, -3.78146e-02,
        4.16330e-02, -3.09924e-02, -4.15530e-02,  5.95500e-02,
       -6.08129e-02,  1.29453e-01,  5.98485e-02, -1.22053e-01,
       -4.65893e-02, -1.76105e-02,  8.36952e-02, -3.43169e-02,
        1.02949e-02, -3.74583e-02,  1.56218e-02,  2.52615e-02,
        3.09000e-02, -6.15672e-03,  1.78421e-02, -7.97611e-03,
        4.08416e-02, -3.61498e-02,  5.93975e-02, -6.13706e-02,
        2.76527e-02, -4.04968e-02,  4.02123e-02,  4.60480e-02,
        3.12373e-02,  4.31485e-02,  4.45423e-03,  9.88624e-05,
       -3.46425e-03, -7.78974e-02,  1.54648e-01, -6.860

In [14]:
def get_dict(file_name):
    my_file = pd.read_csv(file_name, delimiter=' ')
    etof = {}  # the english to french dictionary to be returned
    for i in range(len(my_file)):
        # indexing into the rows.
        en = my_file.loc[i][0]
        fr = my_file.loc[i][1]
        etof[en] = fr

    return etof

In [15]:
en_fr_train = get_dict('../data/en-fr.train.txt')
print('The length of the English to French training dictionary is', len(en_fr_train))
en_fr_test = get_dict('../data/en-fr.test.txt')
print('The length of the English to French test dictionary is', len(en_fr_train))

The length of the English to French training dictionary is 5000
The length of the English to French test dictionary is 5000


## make XR =Y transformation matrix

In [9]:
def get_matrices(en_fr, french_vecs, english_vecs):
    
    X_l = list()
    Y_l = list()
    
    english_words = english_vecs.keys()
    french_words = french_vecs.keys()
    
    for en_word, fr_word in en_fr.items():
        
        if en_word in english_words and fr_word in french_words:
            en_vec = english_vecs[en_word]
            fr_vec = french_vecs[fr_word]
            X_l.append(en_vec)
            Y_l.append(fr_vec)
            
    X =  np.vstack(X_l)
    Y = np.vstack(Y_l)
    
    return X, Y

In [16]:
X_train, Y_train = get_matrices(en_fr_train, fr_embeddings_subset, en_embeddings_subset)

In [18]:
X_train.shape,Y_train.shape

((4932, 300), (4932, 300))

$$ argmin_R(XR-Y)$$
it is what we want for finding translation matrix
The Frobenius norm of a matrix $A$ (assuming it is of dimension $m,n$) is defined as the square root of the sum of the absolute squares of its elements:
$$||A||_F = \sqrt{\sum_{i=1}^{m} \sum_{i=1}^{n}|a_{ij}|^2}$$

In [19]:
def compute_loss(X, Y, R):
    diff = np.dot(X,R)-Y
    element_wise_squared = diff**2
    m = X.shape[0]
    loss = np.sum(element_wise_squared)/m
    return loss

$$\frac{d}{dR}(\frac{1}{m}{||XR-Y||_F}^2)= \frac{2}{m}X^T.(XR-Y)$$ 

In [20]:
def compute_gradient(X, Y, R):
    m = X.shape[0]
    # gradient is X^T(XR - Y) * 2/m
    gradient = np.dot(X.transpose(),np.dot(X,R)-Y)*(2/m)
    return gradient


In [21]:
def align_embeddings(X, Y, train_steps=100, learning_rate=0.0003):
   
    # the number of columns in X is the number of dimensions for a word vector (e.g. 300)
    # R is a square matrix with length equal to the number of dimensions in th  word embedding
    R = np.random.rand(X.shape[1], X.shape[1])

    for i in range(train_steps):
        if i % 25 == 0:
            print(f"loss at iteration {i} is: {compute_loss(X, Y, R):.4f}")
   
        gradient = compute_gradient(X,Y,R)

        # update R by subtracting the learning rate times gradient
        R -=  learning_rate * gradient
    return R

In [22]:
R_train = align_embeddings(X_train, Y_train, train_steps=400, learning_rate=0.8)

loss at iteration 0 is: 956.6748
loss at iteration 25 is: 97.4570
loss at iteration 50 is: 26.8328
loss at iteration 75 is: 9.8324
loss at iteration 100 is: 4.4147
loss at iteration 125 is: 2.3543
loss at iteration 150 is: 1.4657
loss at iteration 175 is: 1.0456
loss at iteration 200 is: 0.8329
loss at iteration 225 is: 0.7197
loss at iteration 250 is: 0.6568
loss at iteration 275 is: 0.6209
loss at iteration 300 is: 0.5996
loss at iteration 325 is: 0.5868
loss at iteration 350 is: 0.5790
loss at iteration 375 is: 0.5740


now we can use k nearest neighbor find similar weord vectors in corpus

In [23]:
def cosine_similarity(A, B):
    
    dot = np.dot(A, B)
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)
    cos = dot / (norma * normb)

    return cos

In [24]:
def nearest_neighbor(v, candidates, k=1):
   
    similarity_l = []

    for row in candidates:
        cos_similarity = cosine_similarity(v,row)
        similarity_l.append(cos_similarity)
        
    sorted_ids = np.argsort(similarity_l)

    k_idx = sorted_ids[-k:]
    return k_idx

In [25]:
X_val, Y_val = get_matrices(en_fr_test, fr_embeddings_subset, en_embeddings_subset)

### for each translation we search whole corpuse to find similar word embedding vector

In [30]:
def test_vocabulary(X, target_lng, R):
    translations = np.dot(X,R)
    num_correct= 0
    for i in range(len(translations)):
        translations_index = nearest_neighbor(translations[i],target_lng)
        
        if translations_index == i:
            num_correct += 1
    
    accuracy = num_correct/len(translations)
    return accuracy

In [31]:
acc = test_vocabulary(X_val, Y_val, R_train)
print(f"accuracy on test set is {acc:.3f}")

accuracy on test set is 0.552
